In [25]:
import os, csv
os.chdir('/scratch/rcj50704/LOL/DLscript')
import pandas as pd
import torch
from transformers import RobertaModel, AutoTokenizer, AutoModel, AutoModelForMaskedLM
import dataloader as dl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import torch.nn as nn
import glob
# plt.rcParams["xtick.top"] = True
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#k-nn
from tqdm import tqdm
import faiss # Or import annoy
! ls -l /work/ah2lab/yuchen/results

total 129350
-rw-r--r-- 1 yz92460 ah2lab  2924835 Jun  5 22:14 test1_1_zeroshot.csv
-rw-r--r-- 1 yz92460 ah2lab  2676315 Jun  5 22:14 test1_2_zeroshot.csv
-rw-r--r-- 1 yz92460 ah2lab  8057361 Jun  5 20:25 test_2_fewshot.csv
-rw-r--r-- 1 yz92460 ah2lab 25188180 Jun  5 20:24 update_Swissbanl_test1.csv
-rw-r--r-- 1 yz92460 ah2lab  6154207 Jun  5 20:23 update_Swissbanl_test2.csv
-rw-r--r-- 1 yz92460 ah2lab 25174299 Jun  5 20:24 update_Swissunba_test1.csv
-rw-r--r-- 1 yz92460 ah2lab  6150728 Jun  5 20:24 update_Swissunba_test2.csv
-rw-r--r-- 1 yz92460 ah2lab 25179866 Jun  5 20:24 update_Unibanl_test1.csv
-rw-r--r-- 1 yz92460 ah2lab  6151792 Jun  5 20:24 update_Unibanl_test2.csv
-rw-r--r-- 1 yz92460 ah2lab 25183110 Jun  5 20:24 update_Uniunba_test1.csv
-rw-r--r-- 1 yz92460 ah2lab  6153309 Jun  5 20:24 update_Uniunba_test2.csv


In [27]:
files = ["update_Swissbanl_test1.csv","update_Swissbanl_test2.csv","update_Swissunba_test1.csv","update_Swissunba_test2.csv","update_Unibanl_test1.csv","update_Unibanl_test2.csv","update_Uniunba_test1.csv","update_Uniunba_test2.csv",]

# for file in files:
df = pd.read_csv(f"/work/ah2lab/yuchen/results/test1_2_zeroshot.csv", usecols=['EC','Predicted EC Number'])
true_labels = df['EC'].tolist()
predicted_labels = df['Predicted EC Number'].tolist()
print(f"{hiclass_score_metrics(true_labels, predicted_labels)}")
    
# df = pd.read_csv(f"/work/ah2lab/yuchen/results/test_2_fewshot.csv")
# df
    
# update_Swissbanl_test1.csv = (0.5389131416528676,  0.1347282854132169,  0.21556525666114706)
# update_Swissbanl_test2.csv = (0.39608533098746423, 0.09902133274686606, 0.1584341323949857)
# update_Swissunba_test1.csv = (0.7512168197099703,  0.18780420492749259, 0.30048672788398817)
# update_Swissunba_test2.csv = (0.47371893556190897, 0.11842973389047724, 0.1894875742247636)
#     update_Unibanl_test1.csv (0.30979978925184404, 0.07744994731296101, 0.12391991570073761)
#     update_Unibanl_test2.csv (0.22850230921486694, 0.057125577303716736,0.09140092368594678)
#     update_Uniunba_test1.csv (0.22489838928195094, 0.056224597320487735,0.08995935571278038)
#     update_Uniunba_test2.csv (0.21464702001319552, 0.05366175500329888, 0.0858588080052782)

(0.9338014042126379, 0.23345035105315948, 0.3735205616850552)


In [6]:
# data_dir="/work/ah2lab/vatsal/Data/final_csv/"
def load_df(files):
    data_dir = "/work/ah2lab/yuchen/results" #"/work/ah2lab/soumya/"
    file_ptrn = f"{files}*"
    all_files = glob.glob(f"{data_dir}/{file_ptrn}")
    print(all_files)
    df= pd.DataFrame()   #load train and test separately
    for filename in all_files:
        temp_df = pd.read_csv(filename, usecols=['EC','Sequence'])
        df = pd.concat([df, temp_df], ignore_index=True)
    return df

# train_df = load_df('SwissprotDatasets/UnbalancedSwissprot/train')
# test_df = load_df('SwissprotDatasets/test')
# train_sequence = train_df['Sequence'].tolist() #[::10]
# train_EC = train_df['EC'].tolist()#[::10]
# test_sequence = test_df['Sequence'].tolist()
# test_EC = test_df['EC'].tolist()
# # test_sequence = test_sequence[::10]
# len(train_sequence), len(train_EC), len(test_sequence), len(test_EC), len(train_df) #179587 5399

(0.5389131416528676, 0.1347282854132169, 0.21556525666114706)

In [1]:
def overlap_score(neighbor_ec, true_ec):
    """
    This function calculates the overlap score between two EC numbers.
    Args:
        ec1 (str): The first EC number (e.g., "1.2.3.4").
        ec2 (str): The second EC number (e.g., "1.2.3.5").
    Returns:
        int: The overlap score, which is the number of matching levels from the beginning.
    """
    levels1 = neighbor_ec.split(".")
    levels2 = true_ec.split(".")
    alphaandbetabet = 0
    alpha = len(levels1)
    beta = len(levels2)
    for i in range(min(len(levels1), len(levels2))):
        if levels1[i] == levels2[i]:
            alphaandbetabet += 1
        else:
            break
    return alphaandbetabet, alpha, beta

def hiclass_score_metrics(true_ecs, neighboring_ecs):
    """
    This function calculates the hierarchical precision (hP) for a set of predicted EC numbers.
    Args:
        true_ecs (list): A list of true EC numbers (e.g., ["1.2.3.4", "2.3.4.5"]).
        neighboring_ecs (list): A list of predicted EC numbers, where each element is a list of k nearest neighbors (e.g., [["1.2.3.5", "2.1.4.3"], ["3.4.5.6", "2.3.4.1"]]).
    Returns:
        float: The average hierarchical precision across all predicted EC numbers. 
    """
    intersection = 0
    alpha = 0
    beta = 0
    for true_ec , neighbor_ecs in zip(true_ecs, neighboring_ecs):
        max_score = 0
        for neighbor_ec in neighbor_ecs:
            alphaandbetabet, a, b = overlap_score(neighbor_ec, true_ec)
            max_score = max(max_score, alphaandbetabet)
        intersection += max_score
        alpha += a
        beta += b
    hP = intersection/alpha
    hR = intersection/beta
    hF = 2*hP*hR/(hP+hR)
    return  hP, hR, hF

In [2]:
def level_accuracy(accuracies, ec , retrieved_ec):
#     print(ec, retrieved_ec)
    for level, accuracy in accuracies.items():
        for r_ec in retrieved_ec:
            if ec.split('.')[:level] == r_ec.split('.')[:level]:
                accuracies[level] += 1
                break
    return accuracies
def top_k_retrieval(train_ec, test_ec, train_embeddings, test_embeddings, k):
    print(f"train_ec={len(train_ec)}, test_ec={len(test_ec)}, train_embeddings={len(train_embeddings)}, test_embeddings={len(test_embeddings)}")
    """
    Performs top-k NN retrieval using RoBERTa encoder embeddings and evaluates
    EC number consistency in top-k retrievals.
    Args:
        model_name (str): Name of the RoBERTa model (e.g., 'roberta-base').
        test_ec (list): List of known EC numbers of test batch.
        train_seqs (list): List of sequences data for which to retrieve similar sequences.
        test_seqs (list): List of sequences data against which retrivals are called.
        k (int): Number of nearest neighbors to retrieve.
    Returns:
        tuple: (accuracy, retrieval_results)
            - accuracy (float): Proportion of top-k retrievals with correct EC numbers.
            - retrieval_results (list of lists): List of retrieved EC numbers for each text data point.
    """
    results = {}
    index = faiss.IndexFlatL2(train_embeddings.size(1))  # Use L2 distance for cosine similarity
    index.add(train_embeddings.cpu().detach().numpy())  # Add embeddings to the index
    for k in K:
        distances, retrieval_indices = index.search(test_embeddings.cpu().detach().numpy(), k)
        retrieval_results = []
        for i, retrieved_idxs in enumerate(retrieval_indices):
    #         print(retrieved_idxs)
            retrieved_ec_numbers = [train_ec[idx] for idx in retrieved_idxs]
            retrieval_results.append(retrieved_ec_numbers)
        accuracies = {} # initialize ec level accuracy
        for level in range(4):
            accuracies.setdefault(level+1, 0)
        for i, retrieved_ec_numbers in enumerate(retrieval_results):
            accuracies = level_accuracy(accuracies, test_ec[i], retrieved_ec_numbers)
        for level, total_correct in accuracies.items():
            accuracies[level] /= len(test_ec)
        #calculate hiclass metrics
        hiclass_metric = hiclass_score_metrics(test_ec, retrieval_results)
        results[k] = accuracies, hiclass_metric
    return results
def get_embedding_stats(embeddings):
    """
    This function extracts class token embedding, minimum, maximum and average 
    for each sequence from the encoder outputs.
    Args:
        embeddings (torch.Tensor): Encoder outputs of shape (batch_size, sequence_length, embedding_dim)
    Returns:
        tuple: A tuple containing three tensors:
            - class_token_embeddings (torch.Tensor): Class token embeddings of shape (batch_size, embedding_dim)
            - min_embeddings (torch.Tensor): Minimum embedding per sequence of shape (batch_size, embedding_dim)
            - max_embeddings (torch.Tensor): Maximum embedding per sequence of shape (batch_size, embedding_dim)
            - avg_embeddings (torch.Tensor): Average embedding per sequence of shape (batch_size, embedding_dim)
    """
    class_token_embeddings = embeddings[:, 0, :]# Extract class token embeddings (assuming it's at index 0 for each sequence)
    min_embeddings, _ = torch.min(embeddings, dim=1)
    max_embeddings, _ = torch.max(embeddings, dim=1)
    avg_embeddings = torch.mean(embeddings, dim=-2) # torch.sum(attention_mask*embeddings, axis=-2)/torch.sum(attention_mask, axis=1)
    return class_token_embeddings, min_embeddings, max_embeddings, avg_embeddings

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
# model = AutoModelForMaskedLM.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
# model = RobertaModel.from_pretrained("models/LOLBERTv4/")
# tokenizer = dl.load_tokenizer("LOLBERTv4/") #FinetunedModel
model = RobertaModel.from_pretrained("models/FinetunedModel/")
tokenizer = dl.load_tokenizer("FinetunedModel/")
# tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species", trust_remote_code=True, local_files_only=True)
# model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species", trust_remote_code=True, local_files_only=True)
model.to(device)
device_ids = [x for x in range(torch.cuda.device_count())]
model = nn.DataParallel(model, device_ids=device_ids)

Some weights of the model checkpoint at models/FinetunedModel/ were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at models/FinetunedModel/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class 

In [10]:
batch_size = 512 #64 #dnabert
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings
    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]
    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}
train_batch = tokenizer(train_sequence, padding='max_length', truncation=True) #lolbert
test_batch = tokenizer(test_sequence, padding='max_length', truncation=True) #lolbert
# train_batch = tokenizer(train_sequence, max_length=128, return_tensors='pt', truncation=True, padding=True) #dnabert
# test_batch = tokenizer(test_sequence, max_length=128, return_tensors='pt', truncation=True, padding=True) #dnabert
# train_batch = tokenizer.batch_encode_plus(train_sequence, max_length=128, return_tensors='pt', truncation=True, padding=True) #nuc
# test_batch = tokenizer.batch_encode_plus(test_sequence, max_length=128, return_tensors='pt', truncation=True, padding=True) #nuc
train_encodings = {'input_ids': torch.tensor(train_batch['input_ids']), 'attention_mask': torch.tensor(train_batch['attention_mask'])} #, 'labels': torch.tensor(batch['input_ids'])
test_encodings = {'input_ids': torch.tensor(test_batch['input_ids']), 'attention_mask': torch.tensor(test_batch['attention_mask'])} #, 'labels': torch.tensor(batch['input_ids'])
train_dataset = Dataset(train_encodings)
test_dataset = Dataset(test_encodings)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
def get_embeddings(loader):
    loop = tqdm(loader, leave=True)
    embeddings = []#torch.Tensor()#.to(device)
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)#     labels = batch['labels'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask) # for lol and dna bert
#             outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True) # for nuc trans
        batch_embeddings = outputs.last_hidden_state.cpu() # LOLBERT
#         batch_embeddings = outputs.hidden_states.cpu() #dnabert
#         batch_embeddings = outputs['hidden_states'][-1].cpu() #for nuc trans
        embeddings.append(batch_embeddings)
    embeddings = torch.cat(embeddings, dim=0)
    return embeddings
train_embeddings = get_embeddings(train_loader)
test_embeddings = get_embeddings(test_loader)

100%|██████████| 48/48 [00:18<00:00,  2.60it/s]


In [ ]:
train_embeddings.shape

# Uniprot balanced

In [8]:
#nuc trans
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's CLASS nearest retrieval stats {1: ({1: 0.39079097891812387, 2: 0.24530151985618565, 3: 0.22499591436509234, 4: 0.1785013891158686}, (0.25989745056381763, 0.25989745056381763, 0.25989745056381763)), 3: ({1: 0.572928583101814, 2: 0.3206406275535218, 3: 0.2739826769079915, 4: 0.20223892792940024}, (0.3424477038731819, 0.3424477038731819, 0.3424477038731819)), 5: ({1: 0.6685324399411668, 2: 0.37203791469194314, 3: 0.3066677561693087, 4: 0.2142506945579343}, (0.39037220134008827, 0.39037220134008827, 0.39037220134008827))}
train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's MIN nearest retrieval stats {1: ({1: 0.420248406602386, 2: 0.28297107370485375, 3: 0.25927439124039875, 4: 0.20603856839352835}, (0.29213310998529174, 0.29213310998529174, 0.29213310998529174)), 3: 

In [12]:
#lolbert
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For models/LOLBERTv4/'s CLASS nearest retrieval stats {1: ({1: 0.38735904559568557, 2: 0.25747671188102633, 3: 0.23627226671024676, 4: 0.1992155580977284}, (0.27008089557117176, 0.27008089557117176, 0.27008089557117176)), 3: ({1: 0.5586288609249878, 2: 0.3242768426213434, 3: 0.27966170942964536, 4: 0.2222176826278804}, (0.3461962739009642, 0.3461962739009642, 0.3461962739009642)), 5: ({1: 0.6501879392057526, 2: 0.3665223075665959, 3: 0.3045432260173231, 4: 0.23042980879228633}, (0.3879208203954895, 0.3879208203954895, 0.3879208203954895))}
train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For models/LOLBERTv4/'s MIN nearest retrieval stats {1: ({1: 0.4296453668900147, 2: 0.3099771204445171, 3: 0.28840496813204775, 4: 0.2448112436672659}, (0.31820967478346135, 0.31820967478346135, 0.31820967478346135)), 3: ({1: 0.5986272266710246, 2: 0.37877921228958983, 3: 0.337269161627717, 4: 0

In [17]:
#finetuned
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For models/FinetunedModel/'s CLASS nearest retrieval stats {1: ({1: 0.5789753227651577, 2: 0.4745464945252492, 3: 0.4577545350547475, 4: 0.3998610884131394}, (0.4777843601895735, 0.4777843601895735, 0.4777843601895735)), 3: ({1: 0.7128207223402516, 2: 0.5489050498447459, 3: 0.5231246935773819, 4: 0.4584082366399739}, (0.5608146756005883, 0.5608146756005883, 0.5608146756005883)), 5: ({1: 0.7759029253145939, 2: 0.590047393364929, 3: 0.5528681157051806, 4: 0.4815737865664324}, (0.6000980552377839, 0.6000980552377839, 0.6000980552377839))}
train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For models/FinetunedModel/'s MIN nearest retrieval stats {1: ({1: 0.5892302663833959, 2: 0.49538323255433897, 3: 0.47589475404477855, 4: 0.41661219153456447}, (0.4942801111292695, 0.4942801111292695, 0.4942801111292695)), 3: ({1: 0.7270795881680013, 2: 0.57431769897042, 3: 0.5412649125674129, 4: 0.4

In [22]:
#dnabert
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's CLASS nearest retrieval stats {1: ({1: 0.3354714822683445, 2: 0.20820395489459062, 3: 0.18773492400719072, 4: 0.15476385030233697}, (0.2215435528681157, 0.2215435528681157, 0.2215435528681157)), 3: ({1: 0.5089883967968623, 2: 0.26295146265729696, 3: 0.21690635724791632, 4: 0.16632619709102794}, (0.28879310344827586, 0.28879310344827586, 0.28879310344827586)), 5: ({1: 0.60818761235496, 2: 0.30756659584899493, 3: 0.24133845399575093, 4: 0.1711881026311489}, (0.3320701912077137, 0.3320701912077137, 0.3320701912077137))}
train_ec=234200, test_ec=24476, train_embeddings=234200, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's MIN nearest retrieval stats {1: ({1: 0.3346543552868116, 2: 0.2076728223565942, 3: 0.18552868115705182, 4: 0.15643896061447948}, (0.22107370485373426, 0.22107370485373426, 0.22107370485373426)), 3: ({1: 0.5081304134662527, 2: 0.26307403170452687, 3: 0.215

# Uniprot unbalanced

In [8]:
#nuc trans
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=278772, test_ec=24476, train_embeddings=278772, test_embeddings=24476
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's CLASS nearest retrieval stats {1: ({1: 0.5469439450890669, 2: 0.44030887399901947, 3: 0.42082039548945904, 4: 0.35974015361987255}, (0.4419533420493545, 0.4419533420493545, 0.4419533420493545)), 3: ({1: 0.7342294492564144, 2: 0.5431443046249387, 3: 0.5033910769733616, 4: 0.4115460042490603}, (0.5480777087759438, 0.5480777087759438, 0.5480777087759438)), 5: ({1: 0.8202320640627554, 2: 0.602631148880536, 3: 0.550049027618892, 4: 0.43802091845072727}, (0.6027332897532276, 0.6027332897532276, 0.6027332897532276))}
train_ec=278772, test_ec=24476, train_embeddings=278772, test_embeddings=24476
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's MIN nearest retrieval stats {1: ({1: 0.6007517568230103, 2: 0.5055156071253473, 3: 0.4866808302010132, 4: 0.41608105899656805}, (0.5022573132864847, 0.5022573132864847, 0.5022573132864847)), 3: ({1: 0.76323

In [26]:
#dnabert
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=557543, test_ec=24476, train_embeddings=557543, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's CLASS nearest retrieval stats {1: ({1: 0.5065370158522634, 2: 0.41130086615460043, 3: 0.39311979081549275, 4: 0.3463392711227325}, (0.41432423598627227, 0.41432423598627227, 0.41432423598627227)), 3: ({1: 0.6646919431279621, 2: 0.48561856512502044, 3: 0.44243340415100507, 4: 0.3767772511848341}, (0.4923802908972054, 0.4923802908972054, 0.4923802908972054)), 5: ({1: 0.7464863539794084, 2: 0.5341150514789998, 3: 0.4747916326197091, 4: 0.3921800947867299}, (0.5368932832162118, 0.5368932832162118, 0.5368932832162118))}
train_ec=557543, test_ec=24476, train_embeddings=557543, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's MIN nearest retrieval stats {1: ({1: 0.5123794737702239, 2: 0.4178378820068639, 3: 0.39945252492237293, 4: 0.3514054584082366}, (0.42026883477692434, 0.42026883477692434, 0.4202688347769243)), 3: ({1: 0.6725772184997548, 2: 0.491910442882824, 3: 0.449910116

In [15]:
#finetune
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=557543, test_ec=24476, train_embeddings=557543, test_embeddings=24476
For models/FinetunedModel/'s CLASS nearest retrieval stats {1: ({1: 0.7243013564307893, 2: 0.6640382415427357, 3: 0.6531704526883477, 4: 0.5842457917960451}, (0.6564389606144795, 0.6564389606144795, 0.6564389606144795)), 3: ({1: 0.8104265402843602, 2: 0.7185814675600588, 3: 0.6995015525412649, 4: 0.6228958980225527}, (0.7128513646020591, 0.7128513646020591, 0.7128513646020591)), 5: ({1: 0.849280928256251, 2: 0.7451380944598791, 3: 0.7200114397777415, 4: 0.638462167020755}, (0.7382231573786566, 0.7382231573786566, 0.7382231573786566))}
train_ec=557543, test_ec=24476, train_embeddings=557543, test_embeddings=24476
For models/FinetunedModel/'s MIN nearest retrieval stats {1: ({1: 0.762828893610067, 2: 0.7116767445661055, 3: 0.7026883477692434, 4: 0.633845399575094}, (0.7027598463801275, 0.7027598463801275, 0.7027598463801275)), 3: ({1: 0.8366154600424907, 2: 0.7549027618891976, 3: 0.7394999182873019, 4: 0.66571

In [9]:
#lolbert
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_EC, test_EC, train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=557543, test_ec=24476, train_embeddings=557543, test_embeddings=24476
For models/LOLBERTv4/'s CLASS nearest retrieval stats {1: ({1: 0.5946641608105899, 2: 0.510745219807158, 3: 0.49415754208203955, 4: 0.441861415263932}, (0.5103570844909299, 0.5103570844909299, 0.5103570844909299)), 3: ({1: 0.7292449746690636, 2: 0.5856349076646511, 3: 0.5518058506291877, 4: 0.4814103611701258}, (0.5870240235332571, 0.5870240235332571, 0.5870240235332571)), 5: ({1: 0.7922863212943292, 2: 0.62828893610067, 3: 0.5830609576728224, 4: 0.49991828730184673}, (0.6258886255924171, 0.6258886255924171, 0.6258886255924171))}
train_ec=557543, test_ec=24476, train_embeddings=557543, test_embeddings=24476
For models/LOLBERTv4/'s MIN nearest retrieval stats {1: ({1: 0.6759274391240399, 2: 0.6088004575911097, 3: 0.5949910116032031, 4: 0.5349730348096094}, (0.6036729857819905, 0.6036729857819905, 0.6036729857819905)), 3: ({1: 0.7821539467233208, 2: 0.6713106716783788, 3: 0.6456937408073214, 4: 0.5744402680176

# swissprot Balanced

In [23]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's CLASS nearest retrieval stats {1: ({1: 0.3315492727569864, 2: 0.12857493054420657, 3: 0.10732962902435038, 4: 0.0539303807811734}, (0.1553460532766792, 0.1553460532766792, 0.1553460532766792)), 3: ({1: 0.5115214904396144, 2: 0.20665141362967807, 3: 0.16126000980552377, 4: 0.07292858310181402}, (0.23809037424415755, 0.23809037424415755, 0.23809037424415755)), 5: ({1: 0.5943373100179767, 2: 0.2555973198235006, 3: 0.19382252001961106, 4: 0.08281581957836248}, (0.2816432423598627, 0.2816432423598627, 0.2816432423598627))}
train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's MIN nearest retrieval stats {1: ({1: 0.36590946233044613, 2: 0.153374734433731, 3: 0.12497957182546168, 4: 0.06635071090047394}, (0.1776536198725282, 0.1776536198725282, 0.17765361987252823)), 3: ({1: 0

In [17]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's CLASS nearest retrieval stats {1: ({1: 0.253350220624285, 2: 0.09862722667102468, 3: 0.07680993626409544, 4: 0.038486680830201014}, (0.11681851609740154, 0.11681851609740154, 0.11681851609740154)), 3: ({1: 0.42625428991665304, 2: 0.16497793757149862, 3: 0.11615460042490602, 4: 0.05417551887563327}, (0.19039058669717274, 0.19039058669717274, 0.19039058669717274)), 5: ({1: 0.5126246118646838, 2: 0.21008334695211636, 3: 0.14340578525902925, 4: 0.06259192678542246}, (0.23217641771531297, 0.23217641771531297, 0.23217641771531297))}
train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's MIN nearest retrieval stats {1: ({1: 0.24052132701421802, 2: 0.08714659258048701, 3: 0.06577872201340089, 4: 0.0298659911750286}, (0.10582815819578363, 0.10582815819578363, 0.10582815819578363)), 3: ({1: 0.4150596502696519, 2: 0.15774636378493218, 3

In [13]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For models/FinetunedModel/'s CLASS nearest retrieval stats {1: ({1: 0.44251511684915834, 2: 0.32537996404641284, 3: 0.305360352998856, 4: 0.25874325870240233}, (0.33299967314920736, 0.33299967314920736, 0.33299967314920736)), 3: ({1: 0.5817944108514463, 2: 0.4074603693413957, 3: 0.37600098055237785, 4: 0.3129187775780356}, (0.41954363458081384, 0.41954363458081384, 0.41954363458081384)), 5: ({1: 0.6546821376041837, 2: 0.4546085961758457, 3: 0.4122405621833633, 4: 0.3328158195783625}, (0.4635867788854388, 0.4635867788854388, 0.4635867788854388))}
train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For models/FinetunedModel/'s MIN nearest retrieval stats {1: ({1: 0.4755270469030887, 2: 0.35263114888053604, 3: 0.331712698153293, 4: 0.2817045268834777}, (0.36039385520509887, 0.36039385520509887, 0.36039385520509887)), 3: ({1: 0.6046331099852917, 2: 0.43070763196600753, 3: 0.397654845563000

In [8]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For models/LOLBERTv4/'s CLASS nearest retrieval stats {1: ({1: 0.2643405785259029, 2: 0.11468377185814675, 3: 0.09286648145121752, 4: 0.061121098218663183}, (0.13325298251348258, 0.13325298251348258, 0.13325298251348258)), 3: ({1: 0.42870567086125183, 2: 0.17678542245464945, 3: 0.12980062101650597, 4: 0.07991501879392057}, (0.20380168328158196, 0.20380168328158196, 0.20380168328158196)), 5: ({1: 0.5132783134499102, 2: 0.2142506945579343, 3: 0.15137277332897534, 4: 0.08759601242033012}, (0.24162444843928746, 0.24162444843928746, 0.24162444843928746))}
train_ec=22280, test_ec=24476, train_embeddings=22280, test_embeddings=24476
For models/LOLBERTv4/'s MIN nearest retrieval stats {1: ({1: 0.29596339271122735, 2: 0.15562183363294657, 3: 0.13453995750939696, 4: 0.09879065206733126}, (0.17122895898022553, 0.17122895898022553, 0.17122895898022553)), 3: ({1: 0.4697663016832816, 2: 0.22712044451707794, 3: 0.17976793593

# swissprot UnBalanced

In [9]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=151314, test_ec=24476, train_embeddings=151314, test_embeddings=24476
For models/LOLBERTv4/'s CLASS nearest retrieval stats {1: ({1: 0.38780846543552866, 2: 0.2613172086942311, 3: 0.24146102304298087, 4: 0.2079996731492074}, (0.274646592580487, 0.274646592580487, 0.274646592580487)), 3: ({1: 0.5561366236313123, 2: 0.35169145285177317, 3: 0.31161137440758296, 4: 0.25988723647654843}, (0.3698316718418042, 0.3698316718418042, 0.3698316718418042)), 5: ({1: 0.6452443209674783, 2: 0.4076237947377022, 3: 0.3516097401536199, 4: 0.28448275862068967}, (0.42224015361987255, 0.42224015361987255, 0.42224015361987255))}
train_ec=151314, test_ec=24476, train_embeddings=151314, test_embeddings=24476
For models/LOLBERTv4/'s MIN nearest retrieval stats {1: ({1: 0.4686223239091355, 2: 0.3577790488641935, 3: 0.341885929073378, 4: 0.30646347442392546}, (0.3686876940676581, 0.3686876940676581, 0.3686876940676581)), 3: ({1: 0.6221196273900964, 2: 0.4451299231900637, 3: 0.4129351201176663, 4: 0.36366

In [12]:
#finetuned
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=151314, test_ec=24476, train_embeddings=151314, test_embeddings=24476
For models/FinetunedModel/'s CLASS nearest retrieval stats {1: ({1: 0.5589965680666775, 2: 0.47184997548619056, 3: 0.4575502533093643, 4: 0.41232227488151657}, (0.47517976793593725, 0.47517976793593725, 0.47517976793593725)), 3: ({1: 0.6759682954731165, 2: 0.5498447458735087, 3: 0.5245138094459879, 4: 0.46935773819251514}, (0.554921147246282, 0.554921147246282, 0.554921147246282)), 5: ({1: 0.7285095603856839, 2: 0.5863703219480307, 3: 0.5526229776107207, 4: 0.48986762542899165}, (0.5893426213433568, 0.5893426213433568, 0.5893426213433568))}
train_ec=151314, test_ec=24476, train_embeddings=151314, test_embeddings=24476
For models/FinetunedModel/'s MIN nearest retrieval stats {1: ({1: 0.6061856512502043, 2: 0.5288445824481124, 3: 0.5158114070926622, 4: 0.4702157215231247}, (0.5302643405785259, 0.5302643405785259, 0.5302643405785259)), 3: ({1: 0.7120444517077954, 2: 0.5992400719071743, 3: 0.5783216211799314, 4:

In [12]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=151314, test_ec=24476, train_embeddings=151314, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's CLASS nearest retrieval stats {1: ({1: 0.33686059813695046, 2: 0.19570191207713677, 3: 0.17617257721849974, 4: 0.1305360352998856}, (0.20981778068311815, 0.20981778068311815, 0.20981778068311815)), 3: ({1: 0.5189164896224874, 2: 0.29069292368033994, 3: 0.2474669063572479, 4: 0.17780683118156562}, (0.30872078771041017, 0.30872078771041017, 0.30872078771041017)), 5: ({1: 0.6122732472626247, 2: 0.3491175028599444, 3: 0.28869096257558424, 4: 0.201340088249714}, (0.3628554502369668, 0.3628554502369668, 0.3628554502369668))}
train_ec=151314, test_ec=24476, train_embeddings=151314, test_embeddings=24476
For zhihan1996/DNABERT-2-117M's MIN nearest retrieval stats {1: ({1: 0.3057689164896225, 2: 0.17535545023696683, 3: 0.15096420983820885, 4: 0.10630822029743422}, (0.1845991992155581, 0.1845991992155581, 0.1845991992155581)), 3: ({1: 0.5096420983820886, 2: 0.27815002451380944, 3: 0.225

In [52]:
train_embedding_stats = get_embedding_stats(train_embeddings)
test_embedding_stats = get_embedding_stats(test_embeddings)
parameters = ['class', 'min', 'max', 'avg']
K = [1, 3, 5]
for train_stat, test_stat, parameter in zip(train_embedding_stats, test_embedding_stats, parameters):
    accuracy = top_k_retrieval(train_df['EC'].tolist(), test_df['EC'].tolist(), train_stat, test_stat, K)
    print(f"For {tokenizer.name_or_path}'s {parameter.upper()} nearest retrieval stats {accuracy}")

train_ec=179587, test_ec=9715, train_embeddings=179587, test_embeddings=9715
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's class nearest retrieval stats {1: ({1: 0.4296448790530108, 2: 0.19948533196088522, 3: 0.18270715388574368, 4: 0.1054040144107051}, (0.2293103448275862, 0.2293103448275862, 0.2293103448275862)), 3: ({1: 0.573031394750386, 2: 0.2655687081832218, 3: 0.23098301595470921, 4: 0.13000514668039115}, (0.29989706639217706, 0.29989706639217706, 0.29989706639217706)), 5: ({1: 0.6545548121461657, 2: 0.31435923829130213, 3: 0.2645393721049923, 4: 0.1422542460113227}, (0.3439269171384457, 0.3439269171384457, 0.3439269171384457))}
train_ec=179587, test_ec=9715, train_embeddings=179587, test_embeddings=9715
For InstaDeepAI/nucleotide-transformer-2.5b-multi-species's min nearest retrieval stats {1: ({1: 0.47133299022130726, 2: 0.24333504889346372, 3: 0.22460113226968606, 4: 0.13268142048378795}, (0.26798764796706126, 0.26798764796706126, 0.26798764796706126)), 3: ({1: 

In [8]:
!free -h && nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:          1.0Ti        46Gi       939Gi       5.7Gi        21Gi       951Gi
Swap:            0B          0B          0B
Sun May 26 18:27:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:01:00.0 Off |                    0 |
| N/A   27C    P0              71W / 500W | 